In [31]:
import panel as pn
import folium as fm
import psycopg
from geopy.geocoders import Nominatim
import pandas as pd
from pymove.visualization import folium as f
import random

pn.extension('tabulator')

In [32]:
height_widgets = 50

In [33]:
conn = psycopg.connect(dbname='fornecer', user='postgres', password='2703705')
cur = conn.cursor()
cur.execute("set schema 'fornecer';")
cur.execute('SELECT * FROM cliente;')
result = cur.fetchall()

In [34]:
button_login = pn.widgets.Button(name='ENTRAR', button_type='primary', height=height_widgets)
button_registration = pn.widgets.Button(name='Registre-se', height=height_widgets)

button_registration_client = pn.widgets.Button(name='Cadastrar', button_type='primary', height=height_widgets, width=620)
button_registration_company = pn.widgets.Button(name='Cadastrar', button_type='primary', height=height_widgets, width=620)
button_registration_unsubscribe = pn.widgets.Button(name='Cancelar cadastro', button_type='success', height=height_widgets, width=620)

button_search_company = pn.widgets.Button(name='BUSCAR', button_type='primary', height=height_widgets)
button_search_product = pn.widgets.Button(name='BUSCAR', button_type='primary', height=height_widgets)
button_search_client = pn.widgets.Button(name='BUSCAR', button_type='primary', height=height_widgets)

button_new_product = pn.widgets.Button(name='Novo produto', button_type='primary', height=height_widgets)
button_add_new_product = pn.widgets.Button(name='Adicionar', button_type='primary', height=height_widgets)
button_delete_product = pn.widgets.Button(name='Remover', button_type='primary')

button_exit = pn.widgets.Button(name='SAIR', button_type='success', height=height_widgets, width=100)

In [35]:
layout_main = pn.Column()
layout_sidebar = pn.Column()

In [36]:
template = pn.template.FastListTemplate(
    site="Fornecer",
    title="Login",
    main=[layout_main],
    sidebar=[],
    main_max_width='385px'
)

In [37]:
widget_name_product = pn.widgets.TextInput(name='Nome', placeholder='Nome')
widget_type_product = pn.widgets.TextInput(name='Tipo', placeholder='Tipo')
widget_price_product = pn.widgets.FloatInput(name='Preço', placeholder='Preço')

template.modal.append('<h2>Novo produto</h2>')
template.modal.append(widget_name_product)
template.modal.append(widget_type_product)
template.modal.append(widget_price_product)
template.modal.append(button_add_new_product)

In [38]:
def modal_add_new_product_open(event):
    template.open_modal()

def modal_add_new_product_close(event):
    template.close_modal()

In [39]:
def update_layout_screen_home_client():
    for x in layout_main:
            layout_main.pop(len(layout_main)-1)

    for x in layout_sidebar:
        layout_sidebar.pop(len(layout_sidebar)-1)
    
    template.header.append(pn.Row(button_exit))
    template.main_max_width=''
    template.title='Cliente'
    template.sidebar.append(layout_sidebar)

def update_layout_screen_home_company():
    for x in layout_main:
            layout_main.pop(len(layout_main)-1)
            
    for x in layout_sidebar:
        layout_sidebar.pop(len(layout_sidebar)-1)
    
    template.header.append(pn.Row(button_exit))
    template.main_max_width=''
    template.title='Empresa'
    template.sidebar.append(layout_sidebar)

def update_tab_my_products(event):
    tab_layout_main_company = layout_main[0][0]

    company_info = tab_layout_main_company[2][1][0].value.cnpj_fornecedor[0]

    cur.execute("SELECT MAX(cod_produto) FROM produto")
    id_product = cur.fetchall()[0][0]+1

    cur.execute("INSERT INTO produto(cod_produto, nome, tipo, preco) VALUES (" +
        "'"+str(id_product)+"'" + ", " +
        "'"+widget_name_product.value+"'" + ", " +
        "'"+widget_type_product.value+"'" + ", " +
        "'"+str(widget_price_product.value)+"'" + ");"
    )
    conn.commit()

    cur.execute("INSERT INTO fornecedor_catalogo_produto(cnpj_fornecedor, cod_produto) VALUES (" + str(company_info) + ", " + str(id_product) + ");")
    conn.commit()

    df_my_products = get_df(
        "SELECT fcp.cnpj_fornecedor, p.cod_produto, p.nome, p.tipo, p.preco FROM produto p, fornecedor_catalogo_produto fcp WHERE fcp.cnpj_fornecedor = " + str(company_info) + " AND fcp.cod_produto = p.cod_produto;",
        ['cnpj_fornecedor', 'cod_produto', 'nome', 'tipo', 'preco']
    )

    table_my_products = pn.widgets.Tabulator(
        df_my_products[['cnpj_fornecedor', 'nome', 'tipo', 'preco']],
        buttons={
            'edit': '#',
            'delete': '#'
        }
    )

    my_products = pn.Column(
        pn.Row('<h3>Meus produtos</h3>'),
        pn.Row(table_my_products),
        pn.Row(button_new_product)
    )

    tab_layout_main_company = layout_main[0][0]
    tab_layout_main_company.pop(2)
    tab_layout_main_company.insert(2, ('Meus produtos', my_products))

    modal_add_new_product_close('close')

def get_df(search, columns):
    cur.execute(search)
    df = pd.DataFrame(cur.fetchall(), columns=columns)

    return df

def filter_df(df, widgets_list):
    for widget in widgets_list:
        if widget.value:
            attribute = widget.placeholder
            value = widget.value
            if type(value) is not list:
                value = [widget.value]

            if attribute == 'preco_produto':
                df = df.loc[df[attribute] <= float(value[0])]
            else:
                df = df[df[attribute].isin(value)]
    
    return df

def filter_df_columns(df, widgets_list, columns):
    for widget in widgets_list:
        if widget.value:
            attribute = widget.placeholder
            value = widget.value
            if type(value) is not list:
                value = [widget.value]
                
            if attribute == 'preco_produto':
                df = df.loc[df[attribute] <= float(value[0])]
            else:
                df = df[df[attribute].isin(value)]

    return df[columns]

In [40]:
def search_company_screen_client(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    df_fornecedor_endereco = get_df(
        "SELECT * FROM fornecedor f, endereco e WHERE f.cep_endereco = e.cep",
        ['cnpj', 'nome', 'cep_endereco', 'email', 'mercado_atuacao', 'senha', 'cep', 'regiao', 'estado', 'cidade', 'lat', 'lon']
    )

    filters_company = [layout_sidebar[0][0][0][1][0], layout_sidebar[0][0][0][2][0], layout_sidebar[0][0][0][3][0], layout_sidebar[0][0][0][4][0], layout_sidebar[0][0][0][5][0]]

    df_fornecedor_endereco_filtered = filter_df_columns(df_fornecedor_endereco, filters_company, ['nome', 'email', 'mercado_atuacao', 'regiao', 'estado', 'cidade'])

    map = f.plot_points(move_data=filter_df(df_fornecedor_endereco, filters_company), slice_tags=['nome', 'mercado_atuacao'], radius=10, color='blue')
    table_companies = pn.widgets.Tabulator(df_fornecedor_endereco_filtered, pagination='remote', page_size=17, width=1120)

    tab_home_main_client = pn.Tabs(
        ('Mapa', pn.panel(map, height=570, width=1120)),
        ('Empresas', pn.panel(table_companies))
    )

    layout_main.append(pn.Row(tab_home_main_client))

def search_product_screen_client(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    df_fornecedor_produto = get_df(
        "SELECT f.cnpj, f.nome, f.email, f.mercado_atuacao, e.regiao, e.estado, e.cidade, e.lat, e.lon, p.nome, p.tipo, p.preco FROM endereco e, fornecedor f, fornecedor_catalogo_produto fcp, produto p WHERE f.cep_endereco = e.cep AND f.cnpj = fcp.cnpj_fornecedor AND fcp.cod_produto = p.cod_produto",
        ['cnpj', 'nome', 'email', 'mercado_atuacao', 'regiao', 'estado', 'cidade', 'lat', 'lon', 'nome_produto', 'tipo_produto', 'preco_produto']
    )

    filters_product = [layout_sidebar[0][0][1][1][0], layout_sidebar[0][0][1][2][0], layout_sidebar[0][0][1][3][0]]

    df_fornecedor_produto_filtered = filter_df_columns(df_fornecedor_produto, filters_product, ['nome', 'email', 'mercado_atuacao', 'regiao', 'estado', 'cidade', 'nome_produto', 'tipo_produto', 'preco_produto'])

    map = f.plot_points(move_data=filter_df(df_fornecedor_produto, filters_product), slice_tags=['nome', 'mercado_atuacao'], radius=10, color='blue')
    table_companies = pn.widgets.Tabulator(df_fornecedor_produto_filtered, pagination='remote', page_size=17, width=1120)

    tab_home_main_client = pn.Tabs(
        ('Mapa', pn.panel(map, height=570, width=1120)),
        ('Empresas', pn.panel(table_companies))
    )

    layout_main.append(pn.Row(tab_home_main_client))

def search_client_screen_company(company_info):
    df_cliente_endereco = get_df(
        "SELECT * FROM cliente c, endereco e WHERE c.cep_endereco = e.cep;",
        ['cpf', 'nome', 'cep_endereco', 'email', 'senha', 'cep', 'regiao', 'estado', 'cidade', 'lat', 'lon']
    )

    df_fornecedor_produto = get_df(
        "SELECT f.cnpj, f.nome, f.email, f.mercado_atuacao, e.regiao, e.estado, e.cidade, e.lat, e.lon, p.nome, p.tipo, p.preco FROM endereco e, fornecedor f, fornecedor_catalogo_produto fcp, produto p WHERE f.cep_endereco = e.cep AND f.cnpj = fcp.cnpj_fornecedor AND fcp.cod_produto = p.cod_produto",
        ['cnpj', 'nome', 'email', 'mercado_atuacao', 'regiao', 'estado', 'cidade', 'lat', 'lon', 'nome_produto', 'tipo_produto', 'preco_produto']
    )

    filters_client = [layout_sidebar[0][0][1][0], layout_sidebar[0][0][2][0], layout_sidebar[0][0][3][0], layout_sidebar[0][0][4][0]]

    df_cliente_endereco_filtered = filter_df_columns(df_cliente_endereco, filters_client, ['nome', 'email', 'regiao', 'estado', 'cidade'])

    map = f.plot_points(move_data=filter_df(df_cliente_endereco, filters_client), slice_tags=['nome'], radius=10, color='blue')
    table_clients = pn.widgets.Tabulator(df_cliente_endereco_filtered, pagination='remote', page_size=15, width=1120)
    

    if len(layout_main) == 0:
        df_my_products = get_df(
            "SELECT fcp.cnpj_fornecedor, p.cod_produto, p.nome, p.tipo, p.preco FROM produto p, fornecedor_catalogo_produto fcp WHERE fcp.cnpj_fornecedor = " + str(company_info[0][0]) + " AND fcp.cod_produto = p.cod_produto;",
            ['cnpj_fornecedor', 'cod_produto', 'nome', 'tipo', 'preco']
        )

        table_my_products = pn.widgets.Tabulator(
            df_my_products[['cnpj_fornecedor', 'nome', 'tipo', 'preco']],
            buttons={
                'delete': 'x'
            }
        )
        
        my_products = pn.Column(
            pn.Row('<h3>Meus produtos</h3>'),
            pn.Row(table_my_products),
            pn.Row(button_new_product)
        )

        tab_home_main_company = pn.Tabs(
            ('Mapa', pn.panel(map, height=570, width=1120)),
            ('Clientes', pn.panel(table_clients)),
            ('Meus produtos', my_products)
        )

        layout_main.append(pn.Row(tab_home_main_company))
    else:
        tab_home_main_company = layout_main[0][0]
        tab_home_main_company.pop(0)
        tab_home_main_company.pop(0)
        tab_home_main_company.insert(0, ('Mapa', pn.panel(map, height=570, width=1120)))
        tab_home_main_company.insert(1, ('Clientes', pn.panel(table_clients)))

In [41]:
df_fornecedor_endereco = get_df(
    "SELECT * FROM fornecedor f, endereco e WHERE f.cep_endereco = e.cep",
    ['cnpj', 'nome', 'cep_endereco', 'email', 'mercado_atuacao', 'senha', 'cep', 'regiao', 'estado', 'cidade', 'lat', 'lon']
)
df_endereco = get_df("SELECT * FROM endereco", ['cep', 'regiao', 'estado', 'cidade', 'lat', 'lon'])
df_fornecedor_produto = get_df(
    "SELECT f.cnpj, f.nome, f.email, f.mercado_atuacao, e.regiao, e.estado, e.cidade, e.lat, e.lon, p.nome, p.tipo, p.preco FROM endereco e, fornecedor f, fornecedor_catalogo_produto fcp, produto p WHERE f.cep_endereco = e.cep AND f.cnpj = fcp.cnpj_fornecedor AND fcp.cod_produto = p.cod_produto",
    ['cnpj', 'nome', 'email', 'mercado_atuacao', 'regiao', 'estado', 'cidade', 'lat', 'lon', 'nome_produto', 'tipo_produto', 'preco_produto']
)

df_cliente_endereco = get_df(
    "SELECT * FROM cliente c, endereco e WHERE c.cep_endereco = e.cep;",
    ['cpf', 'nome', 'cep_endereco', 'email', 'senha', 'cep', 'regiao', 'estado', 'cidade', 'lat', 'lon']
)

In [42]:
def start():
    screen_login('start')

def screen_login(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    if len(template.header) > 0:
        template.header.pop(0)
    
    template.main_max_width = '385px'
    template.title='Login'

    if len(template.sidebar) > 0:
        template.sidebar.pop(0)
    
    layout_main.append(pn.Row('<h2>Faça seu login</h2>'))
    layout_main.append(pn.Row(pn.widgets.TextInput(placeholder='Cpf/Cnpj', height=height_widgets)))
    layout_main.append(pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)))
    layout_main.append(pn.Row(button_login))
    layout_main.append(pn.Row('Ainda não tem uma conta?'))
    layout_main.append(pn.Row(button_registration))

def screen_register(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    template.main_max_width = '700px'
    template.title='Cadastro'

    register_client = pn.Column(
        pn.Row('<h3>Dados do cliente</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cpf', height=height_widgets), pn.widgets.TextInput(placeholder='Email', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Nome', height=height_widgets), pn.widgets.TextInput(placeholder='Sobrenome', height=height_widgets)),
        pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)),
        pn.Row('<h3>Localização</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cep', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Região', height=height_widgets))),
        pn.Row(pn.widgets.TextInput(placeholder='Estado', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Cidade', height=height_widgets))),
        pn.Row(button_registration_client)
    )

    register_company = pn.Column(
        pn.Row('<h3>Dados da empresa</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cnpj', height=height_widgets), pn.widgets.TextInput(placeholder='Email', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Nome da empresa', height=height_widgets), pn.widgets.TextInput(placeholder='Mercado de atuação', height=height_widgets)),
        pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)),
        pn.Row('<h3>Localização</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cep', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Região', height=height_widgets))),
        pn.Row(pn.widgets.TextInput(placeholder='Estado', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Cidade', height=height_widgets))),
        pn.Row(button_registration_company)
    )

    tab_registration = pn.Tabs(
        ('Cliente', register_client),
        ('Empresa', register_company)
    )
    
    layout_main.append(pn.Row('<h2>Crie sua conta</h2>'))
    layout_main.append(tab_registration)
    layout_main.append(button_registration_unsubscribe)

def screen_home_client(client_info):
    update_layout_screen_home_client()

    df_fornecedor_produto = get_df(
        "SELECT f.cnpj, f.nome, f.email, f.mercado_atuacao, e.regiao, e.estado, e.cidade, e.lat, e.lon, p.nome, p.tipo, p.preco FROM endereco e, fornecedor f, fornecedor_catalogo_produto fcp, produto p WHERE f.cep_endereco = e.cep AND f.cnpj = fcp.cnpj_fornecedor AND fcp.cod_produto = p.cod_produto",
        ['cnpj', 'nome', 'email', 'mercado_atuacao', 'regiao', 'estado', 'cidade', 'lat', 'lon', 'nome_produto', 'tipo_produto', 'preco_produto']
    )

    df_fornecedor_endereco = get_df(
        "SELECT * FROM fornecedor f, endereco e WHERE f.cep_endereco = e.cep",
        ['cnpj', 'nome', 'cep_endereco', 'email', 'mercado_atuacao', 'senha', 'cep', 'regiao', 'estado', 'cidade', 'lat', 'lon']
    )

    df_endereco = get_df("SELECT * FROM endereco", ['cep', 'regiao', 'estado', 'cidade', 'lat', 'lon'])

    nomes_empresa = df_fornecedor_endereco.nome.unique()
    mercados_atuacao = df_fornecedor_endereco.mercado_atuacao.unique()
    regioes = df_endereco.regiao.unique()
    estados = df_endereco.estado.unique()
    cidades = df_endereco.cidade.unique()

    search_company = pn.Column(
        pn.Row('<h3>Buscar por empresa</h3>'),
        pn.Row(pn.widgets.MultiChoice(name='Nome da empresa', placeholder='nome', options=list(nomes_empresa), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Mercado de atuação', placeholder='mercado_atuacao', options=list(mercados_atuacao), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Região', placeholder='regiao', options=list(regioes), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Estado', placeholder='estado', options=list(estados), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Cidade', placeholder='cidade', options=list(cidades), solid=False)),
        pn.Row(button_search_company)
    )

    nomes_produto = df_fornecedor_produto.nome_produto.unique()
    tipos_produto = df_fornecedor_produto.tipo_produto.unique()

    search_product = pn.Column(
        pn.Row('<h3>Buscar por produto</h3>'),
        pn.Row(pn.widgets.MultiChoice(name='Nome do produto', placeholder='nome_produto', options=list(nomes_produto), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Tipo', placeholder='tipo_produto', options=list(tipos_produto), solid=False)),
        pn.Row(pn.widgets.FloatInput(name='Preço máximo', placeholder='preco_produto', start=0, end=1000, step=0.01)),
        pn.Row(button_search_product)
    )

    tab_home_sidebar_client = pn.Tabs(
        ('Empresa', search_company),
        ('Produto', search_product)
    )

    layout_sidebar.append(pn.Row(tab_home_sidebar_client))

    search_company_screen_client('search')

def screen_home_company(company_info):
    update_layout_screen_home_company()

    df_cliente_endereco = get_df(
        "SELECT * FROM cliente c, endereco e WHERE c.cep_endereco = e.cep;",
        ['cpf', 'nome', 'cep_endereco', 'email', 'senha', 'cep', 'regiao', 'estado', 'cidade', 'lat', 'lon']
    )

    df_endereco = get_df("SELECT * FROM endereco", ['cep', 'regiao', 'estado', 'cidade', 'lat', 'lon'])

    nomes_cliente = df_cliente_endereco.nome.unique()
    regioes = df_endereco.regiao.unique()
    estados = df_endereco.estado.unique()
    cidades = df_endereco.cidade.unique()

    search_client = pn.Column(
        pn.Row('<h3>Buscar por cliente</h3>'),
        pn.Row(pn.widgets.MultiChoice(name='Nome do cliente', placeholder='nome', options=list(nomes_cliente), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Região', placeholder='regiao', options=list(regioes), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Estado', placeholder='estado', options=list(estados), solid=False)),
        pn.Row(pn.widgets.MultiChoice(name='Cidade', placeholder='cidade', options=list(cidades), solid=False)),
        pn.Row(button_search_client)
    )

    home_sidebar_company = search_client

    layout_sidebar.append(pn.Row(home_sidebar_company))

    search_client_screen_company(company_info)


In [43]:
def check_login_client():
    cur.execute('SELECT cod_cliente, senha FROM cliente;')
    clients = cur.fetchall()

    login_client = None

    for login in clients:
        cpf = login[0]
        password = login[1]
        layout_main_cpf = layout_main[1][0].value
        layout_main_senha = layout_main[2][0].value
        if str(layout_main_cpf) == str(cpf) and str(layout_main_senha) == str(password):
            login_client = [str(cpf), str(password)]
            break
    
    return login_client

def check_login_company():
    cur.execute('SELECT cnpj, senha FROM fornecedor;')
    companies = cur.fetchall()

    login_company = None

    for login in companies:
        cnpj = login[0]
        password = login[1]
        layout_main_cnpj = layout_main[1][0].value
        layout_main_senha = layout_main[2][0].value
        if str(layout_main_cnpj) == str(cnpj) and str(layout_main_senha) == str(password):
            login_company = [str(cnpj), str(password)]
            break
    
    return login_company

def check_account(event):
    login_client = check_login_client()
    login_company = None

    if login_client is None:
        login_company = check_login_company()
    
    if login_client is None and login_company is None:
        if len(layout_main) == 7:
            layout_main.pop(6)
        layout_main.append(pn.Row('<p style="color:red">Login ou senha inválidos</p>'))
    else:
        if login_client is not None:
            cur.execute('SELECT * FROM cliente WHERE cod_cliente = ' + login_client[0])
            client_info = cur.fetchall()
            screen_home_client(client_info)
        else:
            cur.execute('SELECT * FROM fornecedor WHERE cnpj = ' + login_company[0])
            company_info = cur.fetchall()
            screen_home_company(company_info)

def register_client(event):
    cpf = layout_main[1][0][1][0]
    email = layout_main[1][0][1][1]
    nome = layout_main[1][0][2][0]
    sobrenome = layout_main[1][0][2][1]
    senha = layout_main[1][0][3][0]

    cep = layout_main[1][0][5][0]
    regiao = layout_main[1][0][5][1][0]
    estado = layout_main[1][0][6][0]
    cidade = layout_main[1][0][6][1][0]

    clients_data = [cpf, nome, sobrenome, cep, email, senha]
    location_data = [cep, regiao, estado, cidade]

    empty_values_clients = False
    empty_values_location = False

    for data in clients_data:
        if data.value == '':
            empty_values_clients = True
            break
    
    for data in location_data:
        if data.value == '':
            empty_values_location = True
            break
    
    if empty_values_clients or empty_values_location:
        if len(layout_main) > 3:
            layout_main.pop(3)
        layout_main.append(pn.Row('<p style="color:red">Preencha todos os dados</p>'))
    else:
        nome_completo = clients_data[1].value + " " + clients_data[2].value
        cur.execute("INSERT INTO cliente(cod_cliente, nome, cep_endereco, email, senha) VALUES (" +
            clients_data[0].value + ", " + 
            "'"+nome_completo+"'" + ", " +
            clients_data[3].value + ", " +
            "'"+clients_data[4].value+"'" + ", " +
            "'"+clients_data[5].value+"'" + ");"
        )
        conn.commit()

        loc = Nominatim(user_agent="fornecer")
        location = loc.geocode(cidade.value)

        lat = round(location.latitude, 3)
        lon = round(location.longitude, 3)

        cur.execute('INSERT INTO endereco(cep, regiao, estado, cidade, lat, lon) VALUES (' +
            location_data[0].value + ', ' +
            "'"+location_data[1].value+"'" + ', ' +
            "'"+location_data[2].value+"'" + ', ' +
            "'"+location_data[3].value+"'" + ', ' +
            str(lat) + ', ' +
            str(lon) + ');'
        )
        conn.commit()

        cur.execute('SELECT * FROM cliente WHERE cod_cliente = ' + cpf.value)
        client_info = cur.fetchall()
        screen_login('login')

def register_company(event):
    cnpj = layout_main[1][1][1][0]
    email = layout_main[1][1][1][1]
    nome = layout_main[1][1][2][0]
    mercado = layout_main[1][1][2][1]
    senha = layout_main[1][1][3][0]

    cep = layout_main[1][1][5][0]
    regiao = layout_main[1][1][5][1][0]
    estado = layout_main[1][1][6][0]
    cidade = layout_main[1][1][6][1][0]

    company_data = [cnpj, nome, mercado, cep, email, senha]
    location_data = [cep, regiao, estado, cidade]

    empty_values_clients = False
    empty_values_location = False

    for data in company_data:
        if data.value == '':
            empty_values_clients = True
            break
    
    for data in location_data:
        if data.value == '':
            empty_values_location = True
            break
    
    if empty_values_clients or empty_values_location:
        if len(layout_main) > 3:
            layout_main.pop(3)
        layout_main.append(pn.Row('<p style="color:red">Preencha todos os dados</p>'))
    else:
        cur.execute("INSERT INTO fornecedor(cnpj, nome, mercado_atuacao, cep_endereco, email, senha) VALUES (" +
            company_data[0].value + ", " + 
            "'"+company_data[1].value+"'" + ", " +
            "'"+company_data[2].value+"'" + ", " +
            company_data[3].value + ", " +
            "'"+company_data[4].value+"'" + ", "
            "'"+company_data[5].value+"'" + ");"
        )
        conn.commit()

        loc = Nominatim(user_agent="fornecer") 
        location = loc.geocode(cidade.value)

        lat = round(location.latitude, 3) + round(random.uniform(0.001, 0.009), 3)
        lon = round(location.longitude, 3) + round(random.uniform(0.001, 0.009), 3)
        
        cur.execute('INSERT INTO endereco(cep, regiao, estado, cidade, lat, lon) VALUES (' +
            location_data[0].value + ', ' +
            "'"+location_data[1].value+"'" + ', ' +
            "'"+location_data[2].value+"'" + ', ' +
            "'"+location_data[3].value+"'" + ', ' +
            str(lat) + ', ' +
            str(lon) + ');'
        )
        conn.commit()

        cur.execute('SELECT * FROM fornecedor WHERE cnpj = ' + cnpj.value)
        company_info = cur.fetchall()
        screen_login('login')

In [44]:
button_registration.param.watch(screen_register, 'value')
button_registration.value = False

button_registration_unsubscribe.param.watch(screen_login, 'value')
button_registration_unsubscribe.value = False

button_login.param.watch(check_account, 'value')
button_login.value = False

button_exit.param.watch(screen_login, 'value')
button_exit.value = False

button_registration_client.param.watch(register_client, 'value')
button_registration_client.value = False

button_registration_company.param.watch(register_company, 'value')
button_registration_company.value = False

button_search_company.param.watch(search_company_screen_client, 'value')
button_search_company.value = False

button_search_product.param.watch(search_product_screen_client, 'value')
button_search_product.value = False

button_search_client.param.watch(search_client_screen_company, 'value')
button_search_client.value = False

button_new_product.on_click(modal_add_new_product_open)

button_add_new_product.param.watch(update_tab_my_products, 'value')
button_add_new_product.value = False

In [45]:
start()

template.show()

Launching server at http://localhost:53007
